In [44]:
import pandas as pd
import numpy as np 
import google.generativeai as genai
from langchain_core.prompts import PromptTemplate

In [45]:
df = pd.read_csv("MoviesOnStreamingPlatforms_updated.csv")

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df["Age"].unique()

In [ ]:
import json
row_as_dict = df.iloc[0].to_dict()
row_as_dict

In [50]:
row_as_dict["Directors"] = None
row_as_dict["IMDb"] = None

In [ ]:
row_as_json =json.dumps(row_as_dict)
row_as_json

In [52]:
fill_missing_value_prompt = """
You are a highly skilled data analyst with extensive experience in handling and processing movie datasets. Your expertise lies in identifying missing values and providing insights on how to fill those gaps effectively while ensuring data integrity and accuracy.

Your task is to assist in filling missing values in a movie dataset.

The runtime will be in minitues

The Age will be these ['13+', '18+', '7+', 'all', '16+']
**Input Type**: 
A dictionary of key-value pairs where the keys represent column names and the values represent the available data. Some values may be missing (represented as null or None).

**Output Type**: 
A JSON object that contains only the missing values, filled in with appropriate suggestions based on the dataset’s context. You are expected to maintain data consistency and offer plausible replacements where possible.

### Example Input:

{{
    "ID": 1,
    "Title": "Inception",
    "Year": 2010,
    "Age": "13+",
    "IMDb": null,
    "Rotten Tomatoes": "87%",
    "Netflix": 1.0,
    "Hulu": 0,
    "Prime Video": 0,
    "Disney+": 0,
    "Type": 0,
    "Directors": null,
    "Genres": "Action, Adventure, Sci-Fi, Thriller",
    "Country": "United States, United Kingdom",
    "Language": "English, Japanese, French",
    "Runtime": 148.0
}}

###Example Output
{{
    "IMDb": "8.8",
    "Directors": "Christopher Nolan"
}}
Yoy have to check for null values amd return json with actual values with the exact column name and JSON only.

Give output for the following {row_json} ONLY and ONLY in json
JUST THE JSON NOTHING ELSE 
"""

In [53]:
fill_values_template = PromptTemplate(
    input_variables= ["row_json"],
    template=fill_missing_value_prompt
)

In [54]:
import env
genai.configure(api_key = env.api_key)
model = genai.GenerativeModel("gemini-pro")

In [ ]:
df.iloc[0].to_json()

In [56]:
def fill_row(model,row):
    res = model.generate_content(
            contents= fill_values_template.format(
                row_json = row.to_json()
            )
        )
    res = res.text.strip()
    return res

In [ ]:
import json
import time  

for row_index, row in df.iterrows():
    if not  row.isnull().any():
        continue
    filled_row_json = fill_row(model=model, row=row)
    filled_row = json.loads(filled_row_json)

    for col, value in filled_row.items():  
        if col in df.columns:
            df.at[row_index, col] = value  



In [ ]:
df.isnull().sum()